In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
petr4 = yf.Ticker("PETR4.SA")
df = petr4.history(period="max")
df_filtered = df[(df.index >= "2010-01-01") & (df.index <= "2023-12-31")]

In [ ]:
data = df_filtered['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length), 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(scaled_data, seq_length)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
def create_model(learning_rate):
    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        LSTM(units=50),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

learning_rates = [0.001, 0.01]
models = []
histories = []

for lr in learning_rates:
    model = create_model(lr)
    history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.1, verbose=1)
    models.append(model)
    histories.append(history)

In [ ]:
def evaluate_model(model, X, y):
    predictions = model.predict(X)
    mse = np.mean((predictions - y) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(predictions - y))
    return mse, rmse, mae

results = []
for i, model in enumerate(models):
    train_results = evaluate_model(model, X_train, y_train)
    test_results = evaluate_model(model, X_test, y_test)
    results.append((f"Modelo {i+1} (LR: {learning_rates[i]})", train_results, test_results))

In [ ]:
def calculate_accuracy(y_true, y_pred, tolerance=0.05):
    non_zero_indices = y_true != 0
    y_true_non_zero = y_true[non_zero_indices]
    y_pred_non_zero = y_pred[non_zero_indices]
    
    correct_predictions = np.sum(np.abs((y_true_non_zero - y_pred_non_zero) / y_true_non_zero) <= tolerance)
    total_predictions = len(y_true_non_zero)
    
    if total_predictions == 0:
        return 0
    
    accuracy = correct_predictions / total_predictions
    return accuracy

train_accuracy = calculate_accuracy(y_train, models[0].predict(X_train))
test_accuracy = calculate_accuracy(y_test, models[0].predict(X_test))

print(f"Acurácia no treinamento: {train_accuracy * 100:.2f}%")
print(f"Acurácia no teste: {test_accuracy * 100:.2f}%")

In [ ]:
figures = []

fig1 = go.Figure()
for i, history in enumerate(histories):
    fig1.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name=f'Modelo {i+1} - Treino (LR: {learning_rates[i]})'))
    fig1.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name=f'Modelo {i+1} - Validação (LR: {learning_rates[i]})'))
fig1.update_layout(title='Perda do Modelo durante o Treinamento', xaxis_title='Época', yaxis_title='Perda')
figures.append(fig1)

fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=scaler.inverse_transform(y_test.reshape(-1, 1)).flatten(), mode='lines', name='Valores Reais'))
for i, model in enumerate(models):
    fig2.add_trace(go.Scatter(y=scaler.inverse_transform(model.predict(X_test)).flatten(), mode='lines', name=f'Previsões Modelo {i+1}'))
fig2.update_layout(title='Previsões vs Valores Reais', xaxis_title='Tempo', yaxis_title='Preço da Ação')
figures.append(fig2)

metrics = ['MSE', 'RMSE', 'MAE']
fig3 = go.Figure()
for i, (name, train_metrics, test_metrics) in enumerate(results):
    fig3.add_trace(go.Bar(x=metrics, y=train_metrics, name=f'{name} - Treino'))
    fig3.add_trace(go.Bar(x=metrics, y=test_metrics, name=f'{name} - Teste'))
fig3.update_layout(title='Métricas de Erro e Eficiência', xaxis_title='Métricas', yaxis_title='Valor')
figures.append(fig3)

last_sequence = scaled_data[-seq_length:]
next_year_predictions = []

for _ in range(252):
    x_input = last_sequence.reshape((1, seq_length, 1))
    next_price = models[0].predict(x_input)[0]
    next_year_predictions.append(next_price)
    last_sequence = np.append(last_sequence[1:], next_price)

next_year_predictions = scaler.inverse_transform(np.array(next_year_predictions).reshape(-1, 1))

fig4 = go.Figure()
fig4.add_trace(go.Scatter(y=next_year_predictions.flatten(), mode='lines', name='Previsões para o próximo ano'))
fig4.update_layout(title='Previsões das Ações para o Próximo Ano', xaxis_title='Dias', yaxis_title='Preço da Ação')
figures.append(fig4)

for fig in figures:
    fig.show()


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ba64ebc7-14cd-4411-8ff6-86a58d1973cf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>